In [24]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
print('before apt')
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
print('After apt')
print('spark version:', spark_version)
!wget -q https://archive.apache.org/dist/spark/$spark_version/$spark_version-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()
print('All successful')

before apt
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
After apt
spark version: spark-3.0.3
All

In [25]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-02 05:17:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.90MB/s    in 0.2s    

2022-12-02 05:17:56 (4.90 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [26]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [27]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [28]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
vine_reviews_clean_df = df.dropna()

In [29]:
# Create the vine_table DataFrame
vine_df = vine_reviews_clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [30]:
vine_filtered_votes_df = vine_df.filter("total_votes>=20")
vine_filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [31]:
# Filtered for more than 20 total votes, and more than 50% helpful
vine_helpful_by_total_df = vine_filtered_votes_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_by_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [32]:
# Filtered for paid reviews, more than 50% helpful
vine_paid_helpful_df = vine_helpful_by_total_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|
|R1M3LATUI38MK0|          5|           46|         50|   Y|                N|
|R20JZQ3AQVYNAU|          4|          103|        113|   Y|                N|
|R1EHXYTMBO4D28|          1|           56|         61|   Y|     

In [33]:
# Filtered for paid reviews, more than 50% helpful
vine_paid_helpful_df = vine_helpful_by_total_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|
|R1M3LATUI38MK0|          5|           46|         50|   Y|                N|
|R20JZQ3AQVYNAU|          4|          103|        113|   Y|                N|
|R1EHXYTMBO4D28|          1|           56|         61|   Y|     

In [34]:
# Total number of reviews
total_reviews_ct = vine_helpful_by_total_df.count()
total_reviews_ct

121322

In [35]:
# Total paid reviews
total_paid_reviews_df = vine_helpful_by_total_df.filter("vine =='Y'")
total_paid_reviews_ct = total_paid_reviews_df.count()
total_paid_reviews_ct

497

In [36]:
# Total unpaid reviews
total_unpaid_reviews_df = vine_helpful_by_total_df.filter("vine =='N'")
total_unpaid_reviews_ct = total_unpaid_reviews_df.count()
total_unpaid_reviews_ct

120825

In [37]:
# Total five-star reviews
total_five_star_reviews_df = vine_helpful_by_total_df.filter("star_rating == 5")
total_five_star_reviews_ct = total_five_star_reviews_df.count()
total_five_star_reviews_ct

74665

In [38]:
# Total five-star paid reviews
five_star_paid_reviews_df = total_paid_reviews_df.filter("star_rating == 5")
five_star_paid_reviews_ct = five_star_paid_reviews_df.count()
five_star_paid_reviews_ct

220

In [39]:
# Total five-star unpaid reviews
five_star_unpaid_reviews_df = total_unpaid_reviews_df.filter("star_rating < 5")
five_star_unpaid_reviews_ct = five_star_unpaid_reviews_df.count()
five_star_unpaid_reviews_ct

46380

In [40]:
# Five-star paid reviews as percent of total paid reviews
paid_five_star_per_total_paid = (five_star_paid_reviews_ct/total_paid_reviews_ct)*100
round(paid_five_star_per_total_paid, 3)

44.266

In [41]:
# Paid reviews as percent of total five-star reviews
paid_five_star_per_five_star_total = (five_star_paid_reviews_ct/total_five_star_reviews_ct)*100
round(paid_five_star_per_five_star_total, 3)

0.295

In [42]:
# Five-star unpaid reviews as percent of total unpaid reviews
unpaid_five_star_per_total_unpaid = (five_star_unpaid_reviews_ct/total_unpaid_reviews_ct)*100
round(unpaid_five_star_per_total_unpaid, 3)

38.386

In [43]:
# Five-star unpaid reviews as percent of total five-star reviews
unpaid_five_star_per_five_star_total = (five_star_unpaid_reviews_ct/total_five_star_reviews_ct)*100
round(unpaid_five_star_per_five_star_total, 3)

62.117